# Q2: Data Cleaning

**Phase 3:** Data Cleaning & Preprocessing  
**Points: 9 points**

**Focus:** Handle missing data, outliers, validate data types, remove duplicates.

**Lecture Reference:** Lecture 11, Notebook 1 ([`11/demo/01_setup_exploration_cleaning.ipynb`](https://github.com/christopherseaman/datasci_217/blob/main/11/demo/01_setup_exploration_cleaning.ipynb)), Phase 3. Also see Lecture 05 (data cleaning).

---

## Setup

In [198]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Load data from Q1 (or directly from source)
df = pd.read_csv('data/beach_sensors.csv')
# If you saved cleaned data from Q1, you can load it:
# df = pd.read_csv('output/q1_exploration.csv')  # This won't work - load original

---

## Objective

Clean the dataset by handling missing data, outliers, validating data types, and removing duplicates.

**Time Series Note:** For time series data, forward-fill (`ffill()`) is often appropriate for missing values since sensor readings are continuous. However, you may choose other strategies based on your analysis.

---

## Required Artifacts

You must create exactly these 3 files in the `output/` directory:

### 1. `output/q2_cleaned_data.csv`
**Format:** CSV file
**Content:** Cleaned dataset with same structure as original (same columns)
**Requirements:**
- Same columns as original dataset
- Missing values handled (filled, dropped, or imputed)
- Outliers handled (removed, capped, or transformed)
- Data types validated and converted
- Duplicates removed
- **Sanity check:** Dataset should retain most rows after cleaning (at least 1,000 rows). If you're removing more than 50% of data, reconsider your strategy—imputation is usually preferable to dropping rows for this dataset.
- **No index column** (save with `index=False`)

### 2. `output/q2_cleaning_report.txt`
**Format:** Plain text file
**Content:** Detailed report of cleaning operations
**Required information:**
- Rows before cleaning: [number]
- Missing data handling method: [description]
  - Which columns had missing data
  - Method used (drop, forward-fill, impute, etc.)
  - Number of values handled
- Outlier handling: [description]
  - Detection method (IQR, z-scores, domain knowledge)
  - Which columns had outliers
  - Method used (remove, cap, transform)
  - Number of outliers handled
- Duplicates removed: [number]
- Data type conversions: [list any conversions]
- Rows after cleaning: [number]

**Example format:**
```
DATA CLEANING REPORT
====================

Rows before cleaning: 50000

Missing Data Handling:
- Water Temperature: 2500 missing values (5.0%)
  Method: Forward-fill (time series appropriate)
  Result: All missing values filled
  
- Air Temperature: 1500 missing values (3.0%)
  Method: Forward-fill, then median imputation for remaining
  Result: All missing values filled

Outlier Handling:
- Water Temperature: Detected 500 outliers using IQR method (3×IQR)
  Method: Capped at bounds [Q1 - 3×IQR, Q3 + 3×IQR]
  Bounds: [-5.2, 35.8]
  Result: 500 values capped

Duplicates Removed: 0

Data Type Conversions:
- Measurement Timestamp: Converted to datetime64[ns]

Rows after cleaning: 50000
```

### 3. `output/q2_rows_cleaned.txt`
**Format:** Plain text file
**Content:** Single integer number (total rows after cleaning)
**Requirements:**
- Only the number, no text, no labels
- No whitespace before or after
- Example: `50000`

---

## Requirements Checklist

- [ ] Missing data handling strategy chosen and implemented
- [ ] Outliers detected and handled (IQR method, z-scores, or domain knowledge)
- [ ] Data types validated and converted
- [ ] Duplicates identified and removed
- [ ] Cleaning decisions documented in report
- [ ] All 3 required artifacts saved with exact filenames

---

## Your Approach

1. **Handle missing data** - Choose appropriate strategy (drop, forward-fill, impute) based on data characteristics
2. **Detect and handle outliers** - Use IQR method or z-scores; decide whether to remove, cap, or transform
3. **Validate data types** - Ensure numeric and datetime columns are properly typed
4. **Remove duplicates**
5. **Document and save** - Write detailed cleaning report explaining your decisions

---

## Decision Points

- **Missing data:** Should you drop rows, impute values, or forward-fill? Consider: How much data is missing? Is it random or systematic? For time series, forward-fill is often appropriate.
- **Outliers:** Are they errors or valid extreme values? Use IQR method or z-scores to detect, then decide: remove, cap, or transform. Document your reasoning.
- **Data types:** Are numeric columns actually numeric? Are datetime columns properly formatted? Convert as needed.

---

## Checkpoint

After Q2, you should have:
- [ ] Missing data handled
- [ ] Outliers addressed
- [ ] Data types validated
- [ ] Duplicates removed
- [ ] All 3 artifacts saved: `q2_cleaned_data.csv`, `q2_cleaning_report.txt`, `q2_rows_cleaned.txt`

---

**Next:** Continue to `q3_data_wrangling.md` for Data Wrangling.


In [199]:
# Q2_cleaned_data

# inspect rows w missing data

print(df['Air Temperature'])
print(df['Wet Bulb Temperature'])
print(df['Rain Intensity'])
print(df['Total Rain'])
print(df['Precipitation Type'])
print(df['Barometric Pressure'])
print(df['Heading'])


0         16.40
1         17.10
2         18.20
3         17.89
4         19.50
          ...  
196308    -2.50
196309    -3.28
196310    -2.20
196311    -2.39
196312    -1.50
Name: Air Temperature, Length: 196313, dtype: float64
0         12.2
1         11.5
2         12.4
3          NaN
4         13.0
          ... 
196308    -3.7
196309     NaN
196310    -3.3
196311     NaN
196312    -2.8
Name: Wet Bulb Temperature, Length: 196313, dtype: float64
0         0.0
1         0.0
2         0.0
3         NaN
4         0.0
         ... 
196308    0.0
196309    NaN
196310    0.0
196311    NaN
196312    0.0
Name: Rain Intensity, Length: 196313, dtype: float64
0         260.3
1         260.3
2         260.3
3           NaN
4         260.3
          ...  
196308     57.8
196309      NaN
196310     57.8
196311      NaN
196312     57.8
Name: Total Rain, Length: 196313, dtype: float64
0         0.0
1         0.0
2         0.0
3         NaN
4         0.0
         ... 
196308    0.0
196309    NaN
19

In [200]:
num_rows_before = df.shape[0]
print("Number of rows before cleaning:", num_rows_before)

missing_values = df.isnull().sum()
print("Missing values before cleaning:\n", missing_values)

missing_columns = ['Air Temperature', 'Wet Bulb Temperature', 'Rain Intensity',
                   'Total Rain', 'Precipitation Type', 'Barometric Pressure', 'Heading']

missing_with_counts = {}

for col in missing_columns:
    num_missing = df[col].isnull().sum()
    missing_with_counts[col] = num_missing

print("Columns with missing values and their counts:\n", missing_with_counts)

df['Air Temperature'] = df['Air Temperature'].ffill()
df['Wet Bulb Temperature'] = df['Wet Bulb Temperature'].ffill()
df['Rain Intensity'] = df['Rain Intensity'].ffill()
df['Total Rain'] = df['Total Rain'].ffill()
df['Precipitation Type'] = df['Precipitation Type'].ffill()
df['Barometric Pressure'] = df['Barometric Pressure'].ffill()
df['Heading'] = df['Heading'].ffill()

new_num_rows = df.shape[0]

print("Number of rows after cleaning:", new_num_rows)

rows_handled = df.isnull().any(axis=1).sum()

print(df)

print(df.isnull().sum())


Number of rows before cleaning: 196313
Missing values before cleaning:
 Station Name                       0
Measurement Timestamp              0
Air Temperature                   75
Wet Bulb Temperature           75947
Humidity                           0
Rain Intensity                 75947
Interval Rain                      0
Total Rain                     75947
Precipitation Type             75947
Wind Direction                     0
Wind Speed                         0
Maximum Wind Speed                 0
Barometric Pressure              146
Solar Radiation                    0
Heading                        75947
Battery Life                       0
Measurement Timestamp Label        0
Measurement ID                     0
dtype: int64
Columns with missing values and their counts:
 {'Air Temperature': 75, 'Wet Bulb Temperature': 75947, 'Rain Intensity': 75947, 'Total Rain': 75947, 'Precipitation Type': 75947, 'Barometric Pressure': 146, 'Heading': 75947}
Number of rows after clean

In [201]:
def remove_outliers_iqr(dataframe, column):
    mean = dataframe[column].mean()
    std = dataframe[column].std()
    
    lower_bound = mean - 3 * std
    upper_bound = mean + 3 * std

    dataframe[column] = np.clip(dataframe[column], lower_bound, upper_bound)
    return dataframe

numeric_columns = df.select_dtypes(include=[np.number]).columns

columns_with_outliers = {}

for col in numeric_columns:
    mean = df[col].mean()
    std = df[col].std()
    
    lower_bound = mean - 3 * std
    upper_bound = mean + 3 * std
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    if not outliers.empty:
        columns_with_outliers[col] = len(outliers)
    df = remove_outliers_iqr(df, col)

print(df)
print(columns_with_outliers)

                       Station Name   Measurement Timestamp  Air Temperature  \
0       63rd Street Weather Station  09/27/2018 10:00:00 AM            16.40   
1       63rd Street Weather Station  09/27/2018 11:00:00 AM            17.10   
2       63rd Street Weather Station  09/27/2018 01:00:00 PM            18.20   
3            Foster Weather Station  09/27/2018 01:00:00 PM            17.89   
4       63rd Street Weather Station  09/27/2018 03:00:00 PM            19.50   
...                             ...                     ...              ...   
196308   Oak Street Weather Station  12/03/2025 08:00:00 AM            -2.50   
196309       Foster Weather Station  12/03/2025 09:00:00 AM            -3.28   
196310   Oak Street Weather Station  12/03/2025 09:00:00 AM            -2.20   
196311       Foster Weather Station  12/03/2025 10:00:00 AM            -2.39   
196312   Oak Street Weather Station  12/03/2025 10:00:00 AM            -1.50   

        Wet Bulb Temperature  Humidity 

In [202]:
df = df.drop_duplicates()
num_rows_after_drop = df.shape[0]
duplicates_handled = num_rows_before - num_rows_after_drop
print("Number of duplicate rows removed:", duplicates_handled)

Number of duplicate rows removed: 0


In [208]:
df['Measurement Timestamp'] = pd.to_datetime(df['Measurement Timestamp'], format="%m/%d/%Y %I:%M:%S %p")
column_transformed = 'Measurement Timestamp'

print(df)

print(df["Measurement Timestamp"].dtype)

                       Station Name Measurement Timestamp  Air Temperature  \
0       63rd Street Weather Station   2018-09-27 10:00:00            16.40   
1       63rd Street Weather Station   2018-09-27 11:00:00            17.10   
2       63rd Street Weather Station   2018-09-27 13:00:00            18.20   
3            Foster Weather Station   2018-09-27 13:00:00            17.89   
4       63rd Street Weather Station   2018-09-27 15:00:00            19.50   
...                             ...                   ...              ...   
196308   Oak Street Weather Station   2025-12-03 08:00:00            -2.50   
196309       Foster Weather Station   2025-12-03 09:00:00            -3.28   
196310   Oak Street Weather Station   2025-12-03 09:00:00            -2.20   
196311       Foster Weather Station   2025-12-03 10:00:00            -2.39   
196312   Oak Street Weather Station   2025-12-03 10:00:00            -1.50   

        Wet Bulb Temperature  Humidity  Rain Intensity  Interva

In [213]:
df.to_csv('output/q2_cleaned_data.csv', index=False)

print(df["Measurement Timestamp"].dtype)

datetime64[ns]


In [210]:
# q2_report

with open('output/q2_data_cleaning_report.txt', 'w') as f:
    f.write("Data Cleaning Report\n")
    f.write("====================\n\n")
    
    f.write(f"Number of rows before cleaning: {num_rows_before}\n")

    f.write('Missing Data Handling:\n')
    
    for col, missing in missing_with_counts.items():
        f.write(f' - {col}: {missing} ({missing/len(df):.1f})%\n')
        f.write(f'Method: Forward Fill\n')
        f.write('Result: all missing values filled\n')

    f.write("Outlier Handling:\n")
    for col, count in columns_with_outliers.items():
        f.write(f' - {col}: {count} outliers handled using Z-score method (using 3 z-scores)\n')
        f.write('Method: Capped at 3 standard deviations from the mean.\n')
        f.write(f'Number of rows handled: {count}\n')
    f.write(f"Number of duplicate rows removed: {duplicates_handled}\n")
    f.write(f"Data Type Conversions:\n")
    f.write(f" - 'Measurement Timestamp' converted to datetime64[ns]\n")

    f.write(f"Number of rows after cleaning: {num_rows_after_drop}\n")


In [211]:
with open('output/q2_rows_cleaned.txt', 'w') as f:
    f.write(f"{df.shape[0]}")